In [16]:
import pandas as pd
import torch, torch.nn.functional as F

In [23]:
data = pd.read_csv('data/train.csv')

In [25]:
data = data.drop('SPLIT', axis=1)

In [26]:
data.to_csv('data/train.csv', index=False)

In [ ]:
data = pd.read_csv('data/test.csv')

EmptyDataError: No columns to parse from file

In [6]:
characters = ['Br', 'N', ')', 'c', 'o', '6', 's', 'Cl', '=', '2', ']', 'C', 'n', 'O', '4', '1', '#', 'S', 'F', '3', '[', '5', 'H', '(', '-', '[BOS]','[EOS]']
char_to_index = {char: idx for idx, char in enumerate(characters)}
index_to_char = {idx: char for char, idx in char_to_index.items()}
molecules = []
with open('data/train.csv','r') as file:
    for line in file: molecules.append(line.strip())

In [7]:
import torch
import torch.nn.functional as F
class onehotencoder:
    def __init__(self, cti, itc):
        self.cti = cti
        self.itc = itc
        self.len = len(self.cti)

    def encode(self, char):
        return F.one_hot(torch.tensor([self.cti.get(char)]), num_classes = self.len)
    
    def decode(self, vec):
        return self.itc[torch.argmax(vec).item()]
    
    def encode_sequence(self, sequence):
        sequence = sequence.strip()
        tokens = []
        tokens.append('[BOS]')
        i = 0
        while i < len(sequence):
            if i+1 < len(sequence) and sequence[i:i+2] == 'Cl':
                tokens.append('Cl')
                i += 2
            elif i+1 < len(sequence) and sequence[i:i+2] == 'Br':
                tokens.append('Br')
                i += 2
            else:
                tokens.append(sequence[i])
                i += 1
        
        tokens.append('[EOS]')
        indices = [self.cti.get(char) for char in tokens]
        return F.one_hot(torch.tensor(indices), num_classes=self.len)

    def decode_sequence(self, onehot_sequence):
        indices = torch.argmax(onehot_sequence, dim=1).tolist()
        return ''.join([self.itc[idx] for idx in indices])

In [8]:
hotty = onehotencoder(char_to_index,index_to_char)
print(f'This is the molecule before encoding, {molecules[0]}')
print(f'This is the molecule after encoding, {hotty.encode_sequence(molecules[0])}')
print(f'This is the molecule after decoding, {hotty.decode_sequence(hotty.encode_sequence(molecules[0]))}')

This is the molecule before encoding, 
This is the molecule after encoding, tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1]])
This is the molecule after decoding, [BOS][EOS]


In [9]:
fh = open('data/train.csv','r').read()
unique_chars = set(fh)
print(unique_chars)

{'\n'}


In [11]:
print(torch.argmax(hotty.encode('[PAD]')).item())

RuntimeError: Could not infer dtype of NoneType